Подготовка базы данных

In [31]:
import pandas as pd
import threading
import sys
import os

In [32]:
os.mkdir("JSON")

In [33]:
data = pd.read_csv('Raw_data.csv')

In [34]:
data.shape

(371583, 21)

In [35]:
data = data.query("businessman_flag == 0")
data = data.drop(['term','businessman_flag'], axis=1)

In [36]:
data = data.fillna({'start_date': -1, 'fact_close_date': -1, 'card_type_name':-1})

In [37]:
data = data.astype({'start_date': int, 'fact_close_date': int , 'purchase_sum':int ,'purchase_sum':int ,'contract_sum':int , 'current_balance_avg_sum':int , 'current_balance_sum':int, 'current_debit_turn_sum':int, 'current_credit_turn_sum':int})

In [38]:
data.shape

(371583, 19)

In [39]:
data = data.drop_duplicates() # Избавление от дубликатов записей

Поиск валидных записей

In [40]:
listUniqueCardID = data.card_id.unique()

In [41]:
def getlistEqualPartsOfNumberParts(intNub, intParts):
  d, r = divmod(intNub, intParts)
  return [d + (1 if i < r else 0) for i in range(intParts)]

In [42]:
def th_get_ActualUserData(gl_DataFrame, listUniqueCardID, intIndex):
    local_UsersData = pd.DataFrame()
    for i in range(len(listUniqueCardID)):
        if float(i%100) == 0:
            print(i)
            
        strPDCall = f"card_id == '{listUniqueCardID[i]}'" # Запрос

        #f_Data = data.query(strPDCall)
        #print( listUniqueCardID[i] , '-', f_Data.shape)

        local_Data = data.query(strPDCall).drop_duplicates()
        #print( listUniqueCardID[i] , '---', local_Data.shape,'\n')

        if (local_Data.shape[0] != 1):
              
            maxDebet =  local_Data['current_debit_turn_sum'].max()
            maxCredet = local_Data['current_credit_turn_sum'].max()
            maxPurchaseCount = local_Data['purchase_count'].max()
            
            thisDB = local_Data.query(f"current_debit_turn_sum == {maxDebet} & current_credit_turn_sum == {maxCredet} & purchase_count == {maxPurchaseCount}")
            
        else:
            thisDB = local_Data
        
        #print(i, "-", thisDB.shape, "-", listUniqueCardID[i],'\n')

        local_UsersData = pd.concat([local_UsersData, thisDB]) 
           
        #print(local_UsersData.shape)

    local_UsersData.to_csv(f'JSON/New_CSV_{intIndex}.csv')

    gl_DataFrame = pd.concat([gl_DataFrame, local_UsersData]) 
    # print(gl_DataFrame.shape)

    print(f'Thread {intIndex} is done !')
    sys.exit

In [43]:
listParts = getlistEqualPartsOfNumberParts(len(listUniqueCardID),int(len(listUniqueCardID)/700))
listThreads = []
intCount = 0
gl_ActualUserData = pd.DataFrame()

intOldTreadsCount = len(threading.enumerate())

for i in range(len(listParts)):

    if listParts[i] == 0 :
        break
    print(f'Thread {i} from {intCount} to {intCount+listParts[i]} is start - {listParts[i]}')
    listThreads.append( threading.Thread( name=f'Thread_{i}', target=th_get_ActualUserData, args=(gl_ActualUserData, listUniqueCardID[ intCount: intCount+listParts[i] ],i , ) ))
    intCount += listParts[i]+1
    listThreads[i].start()

Thread 0 from 0 to 502 is start - 502
0
Thread 1 from 503 to 1005 is start - 502
0
Thread 2 from 1006 to 1508 is start - 502
0
Thread 3 from 1509 to 2011 is start - 502
0
Thread 4 from 2012 to 2514 is start - 502
0
Thread 5 from 2515 to 3017 is start - 502
0
Thread 6 from 3018 to 3520 is start - 502
0
Thread 7 from 3521 to 4023 is start - 502
0
Thread 8 from 4024 to 4526 is start - 502
0
Thread 9 from 4527 to 5029 is start - 502
0
Thread 10 from 5030 to 5532 is start - 502
0
Thread 11 from 5533 to 6035 is start - 502
0
Thread 12 from 6036 to 6538 is start - 502
0Thread 13 from 6539 to 7041 is start - 502

0Thread 14 from 7042 to 7544 is start - 502

0
Thread 15 from 7545 to 8047 is start - 502
0Thread 16 from 8048 to 8550 is start - 502

0
Thread 17 from 8551 to 9053 is start - 502
0
Thread 18 from 9054 to 9556 is start - 502
0
Thread 19 from 9557 to 10059 is start - 502
0Thread 20 from 10060 to 10562 is start - 502

0
Thread 21 from 10563 to 11065 is start - 502
0
Thread 22 from 11066

In [44]:
while(True):
    if intOldTreadsCount == len(threading.enumerate()) : break

gl_ActualUserData = pd.DataFrame()
count = 0

for i in range(len(listParts)):
    gl_ActualUserData = pd.concat([gl_ActualUserData, pd.read_csv(f'JSON/New_CSV_{i}.csv').drop_duplicates()]) 
    count += pd.read_csv(f'JSON/New_CSV_{i}.csv').shape[0]
gl_ActualUserData.shape

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100100
100
100
100

100
100
100
100
100
100
100
100
100
100
100
100
100
100100
100

100
100
100
100
100
100
100
200
200
200
200
200200

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200200

200
200
200
200
200
200
200
200
200200

200
200
200
200
200
200
200
200
200
200
200
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300300

300
400
400
400
400
400
400
400
400
400
400
400
400
400400

400
400
400
400
400
400400

400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400400

400
400
400
(200, 19)
Thread 44 is done !
500
(196, 19)
Thread 18 is done !
500
500
500
(198, 19)500

Thread 25 is done !
(256, 19)
Thread 30 is done !
500
500
500
500(191, 19)
(210, 19)
Thread 36 is done !
Thread 20 is done !

(181, 19)
Thread 1 is done !
(218, 19)
Thread 24 is done !
(180,

(9482, 20)

In [45]:
gl_ActualUserData.drop(['Unnamed: 0'], axis=1)

,client_id,gender,birth_date,create_date,nonresident_flag,city,contract_sum,product_category_name,card_id,card_type_name,start_date,fact_close_date,purchase_sum,purchase_count,current_balance_avg_sum,current_balance_sum,current_debit_turn_sum,current_credit_turn_sum,card_type
0,d871ef96820b6c7a1ada8e01a772724e,Ж,1982,2006-12-25,R,Юрга,0,Договор на текущий счет для дебетовой карты,429ca8c43cdc5f3d8c9efc9069ee3fa6,-1,-1,-1,0,0,0,0,0,0,dc
1,c79f7d18334216f7bd6c40a05e4a0335,Ж,1980,2015-06-19,R,Октябрьский,0,Договор на текущий счет для дебетовой карты,e3baaf11e34b13c3cbb93fd6e2f6096e,МИР Debit,2019,-1,0,0,12700,1000,15000,7800,dc
2,192b644dd9fc46129e60c2f2f73b851c,Ж,1961,2010-06-04,R,Калтан,0,Договор на текущий счет для дебетовой карты,64c45e9c29fa78a6245ecd2a045c015e,МИР Debit,2018,2019,0,0,0,0,0,0,dc
3,47a1ae7b4e5ce46e69497b59dc87f45e,Ж,1969,2018-02-05,R,Прокопьевск,0,Договор на текущий счет для дебетовой карты,1e61472d87e87e4c7a64d9c14351a5c7,-1,-1,-1,0,0,0,0,0,0,dc
4,c0558c09e6d8967a4f7da4e631af0931,Ж,1992,2018-02-09,R,Уфа,0,Договор на текущий счет для дебетовой карты,29aef8d895f0ba76f3493b58996acd89,МИР Debit,2021,-1,1100,1,400,0,2900,2900,dc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,9acb18399b830419bac9653b52754646,М,1986,2015-05-20,R,Уфа,0,Договор на текущий счет для дебетовой карты,d090cb20c22e2643ec366daf8d26d646,-1,-1,-1,0,0,0,0,0,0,dc
196,8b3206ca8d77aa2446824971f47c531b,Ж,1996,2022-04-28,R,Казань,30000,Кредитная карта,5a0041efbd5832765caf8b9f861bffa3,-1,-1,-1,22700,7,0,0,22700,22700,dc
197,d69d3201104ab393c0638839a7284ad7,М,1953,2018-08-08,R,Пятигорск,200000,Кредитная карта,a78c73e84e893179de17071583cf5084,-1,-1,-1,0,0,0,300,9900,10100,dc
198,a936fb30e2268d2874cdcaa221032915,М,1997,2022-05-12,R,Кемерово,15000,Кредитная карта,e3304eaf55891a175b6abf8030b99da1,-1,-1,-1,15000,33,0,0,15000,15000,dc


In [46]:
gl_ActualUserData.to_csv('RTG_CSV.csv')